In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime

# Add the parent directory to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import modules from our package
from src.utils.data_utils import download_stock_data, get_current_date
from src.strategies.momentum_trading_strategy import momentum_trading_strategy, compute_returns

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

In [2]:
# SSL workaround for Yahoo Finance
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
# Define the output directory
output_dir = "../output"

# Check if the directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")
else:
    print(f"Using existing output directory: {output_dir}")
    
# List files in the output directory
files = os.listdir(output_dir)
csv_files = [f for f in files if f.endswith('.csv')]
image_files = [f for f in files if f.endswith('.png')]

print(f"\nFound {len(csv_files)} CSV files:")
for file in csv_files:
    print(f"  - {file}")
    
print(f"\nFound {len(image_files)} image files:")
for file in image_files[:5]:  # Show only first 5 to avoid cluttering
    print(f"  - {file}")
if len(image_files) > 5:
    print(f"  ... and {len(image_files) - 5} more")

Using existing output directory: ../output

Found 4 CSV files:
  - nvda_trading_strategy_20_100.csv
  - nvda_trading_strategy_50_200.csv
  - NVDA_trading_strategy_5_20.csv
  - nvda_trading_strategy_10_50.csv

Found 13 image files:
  - cumulative_returns_50_200.png
  - nvda_trading_strategy_10_50.png
  - cumulative_returns_20_100.png
  - cumulative_returns_10_50.png
  - buy_sell_signals_5_20.png
  ... and 8 more


In [4]:
# Define the MA pairs we want to analyze
ma_pairs = [(5, 20), (10, 50), (20, 100), (50, 200)]
symbol = "NVDA"

# Storage for results
result_dfs = {}

# Try to load existing result files
for short, long in ma_pairs:
    filename = f"{symbol.lower()}_trading_strategy_{short}_{long}.csv"
    filepath = os.path.join(output_dir, filename)
    
    if os.path.exists(filepath):
        # Load the CSV file
        try:
            df = pd.read_csv(filepath)
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
            result_dfs[(short, long)] = df
            print(f"Loaded existing results for MA pair ({short}, {long})")
        except Exception as e:
            print(f"Error loading {filename}: {e}")
    else:
        print(f"No existing results found for MA pair ({short}, {long})")

Error loading nvda_trading_strategy_5_20.csv: 'Date'
Error loading nvda_trading_strategy_10_50.csv: 'Date'
Error loading nvda_trading_strategy_20_100.csv: 'Date'
Error loading nvda_trading_strategy_50_200.csv: 'Date'


In [5]:
# Check if we need to download data
missing_pairs = [pair for pair in ma_pairs if pair not in result_dfs]

if missing_pairs:
    print(f"Need to generate data for {len(missing_pairs)} MA pairs")
    
    # Download data
    start_date = "2024-01-01"
    end_date = get_current_date()
    
    try:
        # Download data
        df = download_stock_data(symbol, start_date, end_date)
        print(f"Downloaded {len(df)} days of data for {symbol}")
        
        # Generate data for missing pairs
        for short, long in missing_pairs:
            print(f"Generating data for MA pair ({short}, {long})...")
            
            # Calculate signals
            signals = momentum_trading_strategy(df.copy(), short, long)
            
            # Save to CSV
            filename = f"{symbol.lower()}_trading_strategy_{short}_{long}.csv"
            filepath = os.path.join(output_dir, filename)
            signals.to_csv(filepath)
            
            # Store in our dictionary
            result_dfs[(short, long)] = signals
            
            print(f"Data for MA pair ({short}, {long}) saved to {filename}")
    except Exception as e:
        print(f"Error downloading or processing data: {e}")
        print("Creating dummy data for testing instead")
        
        # Create dummy data
        dates = pd.date_range(start=start_date, end=end_date, freq='B')
        price = 100.0
        prices = []
        for _ in range(len(dates)):
            change_percent = np.random.normal(0, 0.02)
            price *= (1 + change_percent)
            prices.append(price)
        
        df = pd.DataFrame({
            'Open': prices,
            'High': [p * (1 + abs(np.random.normal(0, 0.005))) for p in prices],
            'Low': [p * (1 - abs(np.random.normal(0, 0.005))) for p in prices],
            'Close': [p * (1 + np.random.normal(0, 0.002)) for p in prices],
            'Adj Close': [p * (1 + np.random.normal(0, 0.002)) for p in prices],
            'Volume': [int(np.random.normal(1000000, 200000)) for _ in prices]
        }, index=dates)
        
        # Generate data for missing pairs
        for short, long in missing_pairs:
            print(f"Generating dummy data for MA pair ({short}, {long})...")
            
            # Calculate signals
            signals = momentum_trading_strategy(df.copy(), short, long)
            
            # Save to CSV
            filename = f"{symbol.lower()}_trading_strategy_{short}_{long}.csv"
            filepath = os.path.join(output_dir, filename)
            signals.to_csv(filepath)
            
            # Store in our dictionary
            result_dfs[(short, long)] = signals
            
            print(f"Dummy data for MA pair ({short}, {long}) saved to {filename}")
else:
    print("Already have data for all MA pairs")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:997)')))"))


Need to generate data for 4 MA pairs
YF.download() has changed argument auto_adjust default to True
Error downloading data: No module named 'pandas_datareader'
Creating dummy data for testing purposes...
Created dummy data for NVDA with 302 trading days
Downloaded 302 days of data for NVDA
Generating data for MA pair (5, 20)...
Data for MA pair (5, 20) saved to nvda_trading_strategy_5_20.csv
Generating data for MA pair (10, 50)...
Data for MA pair (10, 50) saved to nvda_trading_strategy_10_50.csv
Generating data for MA pair (20, 100)...
Data for MA pair (20, 100) saved to nvda_trading_strategy_20_100.csv
Generating data for MA pair (50, 200)...
Data for MA pair (50, 200) saved to nvda_trading_strategy_50_200.csv


In [6]:
# Calculate returns and metrics for each pair
performance = {}

for pair, df in result_dfs.items():
    # Make sure we have all required columns
    required_columns = ['price', 'short_mavg', 'long_mavg', 'signal', 'positions']
    if not all(col in df.columns for col in required_columns):
        # The CSV may have slightly different column names, try to adapt
        if 'Close' in df.columns and 'price' not in df.columns:
            df['price'] = df['Close']
        if 'Short_MA' in df.columns and 'short_mavg' not in df.columns:
            df['short_mavg'] = df['Short_MA']
        if 'Long_MA' in df.columns and 'long_mavg' not in df.columns:
            df['long_mavg'] = df['Long_MA']
        if 'Signal' in df.columns and 'signal' not in df.columns:
            df['signal'] = df['Signal']
        if 'Position' in df.columns and 'positions' not in df.columns:
            df['positions'] = df['Position']
            
    # Calculate daily returns if not already present
    if 'daily_returns' not in df.columns:
        df['daily_returns'] = df['price'].pct_change().fillna(0)
    
    # Calculate strategy returns if not already present
    if 'strategy_returns' not in df.columns:
        df['strategy_returns'] = df['signal'].shift(1).fillna(0) * df['daily_returns']
    
    # Calculate cumulative returns if not already present
    if 'cumulative_strategy_return' not in df.columns:
        df['cumulative_strategy_return'] = (1 + df['strategy_returns']).cumprod()
    
    # Calculate performance metrics
    final_return = df['cumulative_strategy_return'].iloc[-1] - 1 if len(df) > 0 else 0
    buy_signals = len(df[df['positions'] == 1.0])
    sell_signals = len(df[df['positions'] == -1.0])
    
    # Calculate additional metrics
    annual_factor = 252  # Trading days in a year
    daily_returns = df['strategy_returns'].fillna(0)
    volatility = daily_returns.std() * np.sqrt(annual_factor) if len(daily_returns) > 0 else 0
    sharpe = daily_returns.mean() / daily_returns.std() * np.sqrt(annual_factor) if (len(daily_returns) > 0 and daily_returns.std() > 0) else 0
    max_drawdown = ((1 + daily_returns).cumprod() / (1 + daily_returns).cumprod().cummax() - 1).min() if len(daily_returns) > 0 else 0
    
    # Store metrics
    performance[pair] = {
        'final_return': final_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_drawdown,
        'buy_signals': buy_signals,
        'sell_signals': sell_signals,
        'pair': pair
    }
    
    # Update the dataframe
    result_dfs[pair] = df

# Create a DataFrame with performance metrics
performance_df = pd.DataFrame(performance).T
performance_df.index.names = ['MA Pair']
performance_df.reset_index(inplace=True)
performance_df['short_window'] = performance_df['MA Pair'].apply(lambda x: x[0])
performance_df['long_window'] = performance_df['MA Pair'].apply(lambda x: x[1])

# Sort by final return
performance_df.sort_values('final_return', ascending=False, inplace=True)

performance_df

ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
# Display formatted performance metrics
pd.set_option('display.float_format', '{:.2%}'.format)
display_df = performance_df[['short_window', 'long_window', 'final_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'buy_signals', 'sell_signals']]
display_df.columns = ['Short Window', 'Long Window', 'Return', 'Volatility', 'Sharpe Ratio', 'Max Drawdown', 'Buy Signals', 'Sell Signals']
pd.set_option('display.float_format', '{:.2%}'.format)
display_df.set_index(['Short Window', 'Long Window'], inplace=True)
display_df

In [ ]:
# Reset float format for non-percentage values
pd.reset_option('display.float_format')

In [ ]:
# Create a markdown table for the report
markdown = "# Momentum Trading Strategy Performance Comparison\n\n"
markdown += f"## Stock: {symbol}\n\n"
markdown += "| MA Pair | Return | Volatility | Sharpe | Max Drawdown | Buy Signals | Sell Signals |\n"
markdown += "|---------|--------|------------|--------|--------------|-------------|-------------|\n"

for idx, row in performance_df.iterrows():
    pair = row['MA Pair']
    markdown += f"| ({pair[0]}, {pair[1]}) | {row['final_return']:.2%} | {row['volatility']:.2%} | {row['sharpe_ratio']:.2f} | {row['max_drawdown']:.2%} | {row['buy_signals']} | {row['sell_signals']} |\n"

# Save markdown to file
with open(os.path.join(output_dir, "performance_comparison.md"), "w") as f:
    f.write(markdown)

print("Performance comparison saved to", os.path.join(output_dir, "performance_comparison.md"))
print("\nMarkdown Preview:")
print(markdown)

In [ ]:
# Visualization 1: Comparing Cumulative Returns
plt.figure(figsize=(14, 8))

for pair, df in result_dfs.items():
    if 'cumulative_strategy_return' in df.columns:
        plt.plot(df.index, df['cumulative_strategy_return'], label=f'MA {pair[0]}/{pair[1]}')

plt.axhline(y=1, color='r', linestyle='--', alpha=0.3)
plt.title(f'Comparison of Cumulative Returns for Different MA Pairs - {symbol}')
plt.ylabel('Cumulative Return')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(output_dir, f"{symbol.lower()}_cumulative_returns_comparison.png"))
plt.show()

In [ ]:
# Visualization 2: Drawdown Comparison
plt.figure(figsize=(14, 8))

for pair, df in result_dfs.items():
    if 'cumulative_strategy_return' in df.columns:
        # Calculate drawdown
        cum_returns = df['cumulative_strategy_return']
        running_max = cum_returns.cummax()
        drawdown = (cum_returns / running_max) - 1
        plt.plot(df.index, drawdown, label=f'MA {pair[0]}/{pair[1]}')

plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.title(f'Drawdown Comparison for Different MA Pairs - {symbol}')
plt.ylabel('Drawdown')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(output_dir, f"{symbol.lower()}_drawdown_comparison.png"))
plt.show()

In [ ]:
# Visualization 3: Return vs Volatility Scatterplot
plt.figure(figsize=(10, 8))

returns = performance_df['final_return'].values
volatilities = performance_df['volatility'].values
sharpe_ratios = performance_df['sharpe_ratio'].values
labels = [f"({short}, {long})" for short, long in performance_df['MA Pair']]

# Create scatter plot
plt.scatter(volatilities, returns, s=200, c=sharpe_ratios, cmap='viridis', alpha=0.7)

# Add colorbar for Sharpe ratio
cbar = plt.colorbar()
cbar.set_label('Sharpe Ratio')

# Add labels to each point
for i, label in enumerate(labels):
    plt.annotate(label, (volatilities[i], returns[i]), 
                 xytext=(7, 7), textcoords='offset points',
                 fontsize=8, alpha=0.8)

# Add reference lines
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='r', linestyle='--', alpha=0.3)

plt.title(f'Risk-Return Profile of Different MA Pairs - {symbol}')
plt.xlabel('Volatility (Annualized)')
plt.ylabel('Return')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, f"{symbol.lower()}_risk_return_profile.png"))
plt.show()

In [ ]:
# Visualization 4: Trading Activity Heatmap

# Create a DataFrame with trading activity counts
trading_activity = pd.DataFrame(index=pd.DatetimeIndex([]))

for pair, df in result_dfs.items():
    # Create a Series with 1 for buy signals, -1 for sell signals, 0 otherwise
    pair_label = f"({pair[0]}, {pair[1]})"
    signals = pd.Series(0, index=df.index)
    signals[df['positions'] == 1.0] = 1  # Buy
    signals[df['positions'] == -1.0] = -1  # Sell
    # Only keep non-zero values
    signals = signals[signals != 0]
    if not signals.empty:
        trading_activity[pair_label] = signals

# If we have trading activity data
if not trading_activity.empty:
    # Resample to monthly frequency for better visualization
    monthly_activity = trading_activity.resample('M').count()
    
    plt.figure(figsize=(14, 8))
    sns.heatmap(monthly_activity.T, cmap='YlGnBu', linewidths=0.5, 
                cbar_kws={'label': 'Number of Trades per Month'})
    plt.title(f'Monthly Trading Activity by MA Pair - {symbol}')
    plt.ylabel('MA Pair')
    plt.xlabel('Month')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{symbol.lower()}_trading_activity_heatmap.png"))
    plt.show()
else:
    print("No trading activity data available for heatmap visualization")

In [ ]:
# Visualization 5: Comparative Performance Metrics

# Prepare data for bar charts
metrics = ['final_return', 'sharpe_ratio', 'max_drawdown']
metric_labels = ['Return', 'Sharpe Ratio', 'Max Drawdown']

# Create a figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, (metric, label) in enumerate(zip(metrics, metric_labels)):
    # Sort by the current metric
    if metric == 'max_drawdown':
        # For drawdown, less negative is better
        sorted_df = performance_df.sort_values(metric, ascending=False)
    else:
        # For others, higher is better
        sorted_df = performance_df.sort_values(metric, ascending=True)
    
    # Create bar colors based on values
    if metric == 'max_drawdown':
        # For drawdown, more negative values should be more red
        colors = plt.cm.RdYlGn(np.linspace(0, 1, len(sorted_df)))
    else:
        # For others, higher values should be more green
        colors = plt.cm.RdYlGn(np.linspace(0, 1, len(sorted_df)))
    
    # Create bar chart
    bars = axes[i].bar(
        [f"({short}, {long})" for short, long in sorted_df['MA Pair']], 
        sorted_df[metric],
        color=colors
    )
    
    # Add value labels on the bars
    for bar in bars:
        height = bar.get_height()
        if metric == 'sharpe_ratio':
            label_text = f"{height:.2f}"
        else:
            label_text = f"{height:.2%}"
        axes[i].text(
            bar.get_x() + bar.get_width()/2., 
            height * (1.05 if height > 0 else 0.95),
            label_text,
            ha='center', va='bottom' if height > 0 else 'top',
            fontsize=8, rotation=90
        )
    
    # Configure axis
    axes[i].set_title(label)
    axes[i].tick_params(axis='x', rotation=90)
    axes[i].grid(axis='y', alpha=0.3)
    
    # For drawdown, invert y-axis to show more negative values at the bottom
    if metric == 'max_drawdown':
        axes[i].invert_yaxis()

plt.tight_layout()
plt.savefig(os.path.join(output_dir, f"{symbol.lower()}_comparative_metrics.png"))
plt.show()

In [ ]:
# Visualization 6: Moving Average Crossover Points for Best Pair

# Find best pair based on return
best_pair = performance_df.iloc[0]['MA Pair']
best_df = result_dfs[best_pair]

plt.figure(figsize=(14, 8))

# Plot price
plt.plot(best_df.index, best_df['price'], label='Price', color='grey', alpha=0.5)

# Plot moving averages
plt.plot(best_df.index, best_df['short_mavg'], label=f'Short MA ({best_pair[0]})', color='blue')
plt.plot(best_df.index, best_df['long_mavg'], label=f'Long MA ({best_pair[1]})', color='green')

# Highlight crossover points
for i in range(1, len(best_df)):
    # Check for crossover: short > long (buy)
    if (best_df['short_mavg'].iloc[i-1] <= best_df['long_mavg'].iloc[i-1] and 
        best_df['short_mavg'].iloc[i] > best_df['long_mavg'].iloc[i]):
        plt.scatter(best_df.index[i], best_df['short_mavg'].iloc[i], 
                   s=100, marker='^', color='green', edgecolors='black')
    
    # Check for crossover: short < long (sell)
    elif (best_df['short_mavg'].iloc[i-1] >= best_df['long_mavg'].iloc[i-1] and 
          best_df['short_mavg'].iloc[i] < best_df['long_mavg'].iloc[i]):
        plt.scatter(best_df.index[i], best_df['short_mavg'].iloc[i], 
                   s=100, marker='v', color='red', edgecolors='black')

plt.title(f'Moving Average Crossovers for Best Pair MA {best_pair[0]}/{best_pair[1]} - {symbol}')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, f"{symbol.lower()}_best_pair_crossovers.png"))
plt.show()

In [ ]:
# Create a summary of our findings
summary = f"""# Momentum Trading Strategy Analysis Summary

## Stock: {symbol}

### Best Performing Strategy
- **MA Pair**: ({best_pair[0]}, {best_pair[1]})
- **Return**: {performance_df.iloc[0]['final_return']:.2%}
- **Sharpe Ratio**: {performance_df.iloc[0]['sharpe_ratio']:.2f}
- **Max Drawdown**: {performance_df.iloc[0]['max_drawdown']:.2%}
- **Number of Trades**: {performance_df.iloc[0]['buy_signals'] + performance_df.iloc[0]['sell_signals']}

### Key Insights
- The best performing MA pair generated a return of {performance_df.iloc[0]['final_return']:.2%} over the analysis period.
- Shorter MA pairs ({ma_pairs[0][0]}, {ma_pairs[0][1]}) tend to generate more trading signals compared to longer MA pairs ({ma_pairs[-1][0]}, {ma_pairs[-1][1]}).
- The tradeoff between return and volatility varies across different MA pairs, with some providing better risk-adjusted returns than others.
- Maximum drawdown ranges from {performance_df['max_drawdown'].min():.2%} to {performance_df['max_drawdown'].max():.2%}, highlighting the importance of risk management.

### Recommendations
- Consider the ({best_pair[0]}, {best_pair[1]}) MA pair for momentum trading with {symbol} based on historical performance.
- Take into account transaction costs and slippage, which could significantly impact actual returns, especially for strategies with higher trading frequency.
- Combine MA crossover signals with other technical indicators to filter out false signals and improve performance.
- Regularly reassess the optimal MA pairs as market conditions change over time.
"""

# Save summary
with open(os.path.join(output_dir, "analysis_summary.md"), "w") as f:
    f.write(summary)

print("Analysis summary saved to", os.path.join(output_dir, "analysis_summary.md"))
print("\nSummary Preview:")
print(summary)

In [ ]:
# This notebook has analyzed and visualized momentum trading strategies using different moving average pairs.
# We've compared their performance, identified the best parameters, and gained insights into the risk-return tradeoffs.
# This analysis can help traders make more informed decisions about which moving average parameters to use for momentum trading.
print("Analysis complete! All visualizations and reports have been saved to the output directory.")